<a href="https://colab.research.google.com/github/tthaischagas/bigdata_airlines/blob/main/teste_eleflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!wget https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvzf spark-3.3.0-bin-hadoop3.tgz
!pip install findspark

--2022-10-13 23:43:39--  https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299321244 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop3.tgz.1’

spark-3.3.0-bin-had 100%[===================>] 285.45M  22.9MB/s    in 12s     

2022-10-13 23:43:51 (24.6 MB/s) - ‘spark-3.3.0-bin-hadoop3.tgz.1’ saved [299321244/299321244]

spark-3.3.0-bin-hadoop3/
spark-3.3.0-bin-hadoop3/NOTICE
spark-3.3.0-bin-hadoop3/kubernetes/
spark-3.3.0-bin-hadoop3/kubernetes/tests/
spark-3.3.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.3.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.3.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.3.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.3.0-bin-hado

In [9]:
import datetime
import pandas as pd
import os
import glob
import findspark
import json
import requests
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
findspark.init()

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import exp
from functools import reduce 
from pyspark.sql import DataFrame as dfpysp
from pyspark.sql import functions as func
from pyspark.sql.functions import regexp_replace
from pyspark.shell import spark
from google.colab import drive
sc= spark.sparkContext
spark.conf.set("spark.rapids.sql.enabled", False)
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.7.14 (default, Sep  8 2022 00:06:44)
Spark context Web UI available at http://482d5e868f67:4040
Spark context available as 'sc' (master = local[*], app id = local-1665704651257).
SparkSession available as 'spark'.


In [11]:
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#carregar dados de vra
folderName = "/content/drive/MyDrive/teste_eleflow/VRA/"

schema = StructType([\
   StructField("ChegadaPrevista",StringType(),True), \
   StructField("ChegadaReal",StringType(),True), \
   StructField("CódigoAutorização",StringType(),True), \
   StructField("CódigoTipoLinha", StringType(), True), \
   StructField("CódigoJustificativa", StringType(), True), \
   StructField("ICAOAeródromoOrigem", StringType(), True), \
   StructField("ICAOAeródromoDestino", StringType(), True), \
   StructField("ICAOEmpresaAérea", StringType(), True), \
   StructField("NúmeroVoo", StringType(), True), \
   StructField("PartidaPrevista", DateType(), True), \
   StructField("PartidaReal", DateType(), True), \
   StructField("SituaçãoVoo", StringType(), True) \
 ])

df_vra = spark.read.json(folderName, schema, multiLine=True)

In [13]:
#snake case campos vra
df_vra = df_vra.withColumnRenamed('ChegadaPrevista','chegada_prevista') \
        .withColumnRenamed('ChegadaReal','chegada_real') \
        .withColumnRenamed('CódigoAutorização','codigo_autorizacao') \
        .withColumnRenamed('CódigoTipoLinha','codigo_tipo_linha') \
        .withColumnRenamed('CódigoJustificativa','codigo_justificativa') \
        .withColumnRenamed('ICAOAeródromoOrigem','icao_aerodromo_origem') \
        .withColumnRenamed('ICAOAeródromoDestino','icao_aerodromo_destino') \
        .withColumnRenamed('ICAOEmpresaAérea','icao_empresa_aerea') \
        .withColumnRenamed('NúmeroVoo','numero_voo') \
        .withColumnRenamed('PartidaPrevista','partida_prevista') \
        .withColumnRenamed('PartidaReal','partida_real') \
        .withColumnRenamed('SituaçãoVoo','situacao_voo')

df_vra.show()
df_vra.printSchema()

#gravar resultado em parquet vra
df_vra.write.parquet("/content/drive/MyDrive/teste_eleflow/result_VRA/")

+-------------------+-------------------+------------------+-----------------+--------------------+---------------------+----------------------+------------------+----------+----------------+------------+------------+
|   chegada_prevista|       chegada_real|codigo_autorizacao|codigo_tipo_linha|codigo_justificativa|icao_aerodromo_origem|icao_aerodromo_destino|icao_empresa_aerea|numero_voo|partida_prevista|partida_real|situacao_voo|
+-------------------+-------------------+------------------+-----------------+--------------------+---------------------+----------------------+------------------+----------+----------------+------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:24:00|                 0|                X|                 N/A|                 SBGR|                  KORD|               UAL|      0844|      2021-11-11|  2021-11-11|   REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:05:00|                 0|                X|                 N/A|                 SBGR|       

AnalysisException: ignored

In [ ]:
#carrega arquivos aircia
folderName = "/content/drive/MyDrive/teste_eleflow/AIR_CIA/"

#estruturando os datatypes e nomes dos campos
schema = StructType([ \
    StructField("razao_social", StringType(), True), \
    StructField("icao_iata", StringType(), True), \
    StructField("cnpj",StringType(),True), \
    StructField("atividades_aeras", StringType(), True), \
    StructField("endereco_sede", StringType(), True), \
    StructField("telefone", StringType(), True), \
    StructField("email", StringType(), True), \
    StructField("decisao_operacional", StringType(), True), \
    StructField("data_decisao_operacional", DateType(),True), \
    StructField("validade_operacional", DateType(),True) \
  ])

#lendo arquivos csv e definindo o formato de date
df_aircia = spark.read.csv(folderName, schema, sep=";", multiLine=True, header=True, dateFormat="dd/MM/yyyy")

#separando a coluna icao iata
df_aircia = df_aircia.withColumn("tmp",split(col("icao_iata"),"\s+")) \
            .withColumn("icao",element_at(col("tmp"),1)) \
            .withColumn("iata",element_at(col("tmp"),2)) \
            .drop('tmp').drop('icao_iata')


df_aircia.show()
#df_aircia.printSchema()

#gravar resultado em parquet aircia
df_aircia.write.parquet("/content/drive/MyDrive/teste_eleflow/result_AIRCIA/")

+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|        razao_social|              cnpj|    atividades_aeras|       endereco_sede|            telefone|               email| decisao_operacional|data_decisao_operacional|validade_operacional|icao|iata|
+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|ABSA - AEROLINHAS...|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              2015-04-22|          2025-04-23| LTG|  M3|
|AEROSUL TÁXI AÉRE...|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              2021-02-10|                n

In [ ]:
#carrega arquivos aircia
# folderName = "/content/drive/MyDrive/teste_eleflow/AIR_CIA/"

# fileExtension = 'csv'
# delimiter = ';'
# headerExists = 'True'

# df_aircia = pd.DataFrame()

# for arquivo in os.listdir(folderName):
#     caminho_arquivo = folderName + "/" + arquivo 
#     print(caminho_arquivo)
#     df = pd.read_csv(caminho_arquivo,sep=';',header=0,low_memory=False,encoding= 'utf-8') 
#     df_aircia = df_aircia.append(df)

In [ ]:
# schema = StructType([ \
#    StructField("razao_social",StringType(),True), \
#    StructField("icao_iata",StringType(),True), \
#    StructField("cnpj",StringType(),True), \
#    StructField("atividades_aeras", StringType(), True), \
#    StructField("endereco", StringType(), True), \
#    StructField("sede", StringType(), True), \
#    StructField("telefone", StringType(), True), \
#    StructField("email", StringType(), True), \
#    StructField("decisao_operacional", StringType(), True), \
#    StructField("data_decisao_operacional", DateType(), True), \
#    StructField("validade_operacional", DateType(), True) \
#  ])

# df_aircia.rename(columns={'Razão Social' : 'razao_social','ICAO IATA' : 'icao_iata','CNPJ' : 'cnpj'
# ,'Atividades Aéreas' : 'atividades_aeras','Endereço Sede' : 'endereco_sede','Telefone' : 'telefone'
# ,'E-Mail' : 'email','Decisão Operacional' : 'decisao_operacional','Data Decisão Operacional' : 'data_decisao_operacional','Validade Operacional' : 'validade_operacional'},inplace=True)



# #df_aircia_FINAL = spark.createDataFrame(df_aircia.astype(str))
# #df_aircia = spark.read.csv(folderName, schema, sep=";", multiLine=True,header=True, date_format="dd/MM/yyyy")

In [ ]:
# df_aircia = df_aircia.withColumn("tmp",split(col("icao_iata"),"\s+")) \
# .withColumn("icao",element_at(col("tmp"),1)) \
# .withColumn("icao",element_at(col("tmp"),2)) \
# .drop(col("tmp")) \
# .drop(col("icao_iata"))

In [ ]:
#criacao de tabela aerodromos atraves da API atraves do codigo icao do vra
icaos_origem = [i.icao_aerodromo_origem for i in df_vra.select('icao_aerodromo_origem').distinct().collect()]
icaos_destino = [i.icao_aerodromo_destino for i in df_vra.select('icao_aerodromo_destino').distinct().collect()]
icaos = list(set(icaos_origem + icaos_destino))
#busca dados na API aerodromos
def get_icao(icao):
  url = "https://airport-info.p.rapidapi.com/airport"
  headers = {
    "X-RapidAPI-Host": "airport-info.p.rapidapi.com",
    "X-RapidAPI-Key": "df38e36e4emshf6f9847f9ece0f9p1c1d58jsn18072b3e2ebb"
  }
  querystring = {"icao":icao}
  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.text

lista = [get_icao(icao) for icao in icaos ]
lista = [x for x in lista if 'No airport found' not in x]
print(*lista)

tsRDD = sc.parallelize(lista)
df_aerodromo = spark.read.option('multiline', "true").json(tsRDD)
df_aerodromo.show()
df_aerodromo.write.parquet("/content/drive/MyDrive/teste_eleflow/AERODROMOS/")

{"id":7818,"iata":"UNA","icao":"SBTC","name":"Una-Comandatuba Airport","location":"Una / Comandatuba Island, Bahia, Brazil","street_number":"","street":"","city":"","county":"Una","state":"State of Bahia","country_iso":"BR","country":"Brazil","postal_code":"45690-000","phone":"","latitude":-15.357702,"longitude":-38.9957,"uct":-180,"website":""}
 {"id":2886,"iata":"HHN","icao":"EDFH","name":"Frankfurt–Hahn Airport","location":"Hahn, Rhineland-Palatinate, Germany","street_number":"","street":"","city":"Hahn-Flughafen","county":"","state":"Rheinland-Pfalz","country_iso":"DE","country":"Germany","postal_code":"55483","phone":"+49 6543 509113","latitude":49.9429,"longitude":7.262953,"uct":120,"website":"http://www.hahn-airport.de/"}
 {"id":8322,"iata":"XAP","icao":"SBCH","name":"Serafin Enoss Bertaso Airport","location":"Chapecó, Santa Catarina, Brazil","street_number":"4535D","street":"Acesso Florenal Ribeiro","city":"","county":"Chapecó","state":"Santa Catarina","country_iso":"BR","count

In [ ]:
#rota mais utilizada por cia aerea
df_aircia.createOrReplaceTempView("aircia")
df_vra.createOrReplaceTempView("vra")
df_aerodromo.createOrReplaceTempView("aerodromo")
spark.sql("SELECT count(*) as qtd , icao_empresa_aerea , icao_aerodromo_origem,icao_aerodromo_destino from vra where situacao_voo = 'REALIZADO' group by icao_empresa_aerea,icao_aerodromo_origem,icao_aerodromo_destino").createOrReplaceTempView('qtd_empresa')
spark.sql("select max(qtd) maximo ,icao_empresa_aerea from qtd_empresa group by icao_empresa_aerea ").createOrReplaceTempView('max_empresa')
spark.sql("select qe.icao_empresa_aerea, qe.icao_aerodromo_origem, qe.icao_aerodromo_destino from max_empresa me inner join qtd_empresa qe on (me.icao_empresa_aerea = qe.icao_empresa_aerea and me.maximo = qe.qtd)").createOrReplaceTempView('jj')

#resultado final
spark.sql("SELECT DISTINCT A.RAZAO_SOCIAL, AE_ORIGEM.NAME AS AEROPORTO_ORIGEM , AE_ORIGEM.ICAO AS ICAO_ORIGEM , AE_ORIGEM.STATE AS ESTADO_ORIGEM ,AE_DESTINO.NAME AS AEROPORTO_DESTINO , AE_DESTINO.ICAO AS ICAO_AEROPORTO_DESTINO , AE_DESTINO.STATE AS ESTADO_DESTINO FROM JJ J INNER JOIN AIRCIA A ON (J.icao_empresa_aerea = A.ICAO) LEFT JOIN AERODROMO AE_ORIGEM ON (J.ICAO_AERODROMO_ORIGEM = AE_ORIGEM.ICAO) LEFT JOIN AERODROMO AE_DESTINO ON (J.ICAO_AERODROMO_DESTINO = AE_DESTINO.ICAO)  order by 1").show()


+--------------------+--------------------+-----------+-----------------+--------------------+----------------------+--------------------+
|        RAZAO_SOCIAL|    AEROPORTO_ORIGEM|ICAO_ORIGEM|    ESTADO_ORIGEM|   AEROPORTO_DESTINO|ICAO_AEROPORTO_DESTINO|      ESTADO_DESTINO|
+--------------------+--------------------+-----------+-----------------+--------------------+----------------------+--------------------+
|ABSA - AEROLINHAS...|Mariscal Sucre In...|       SEQM|        Pichincha|Miami Internation...|                  KMIA|             Florida|
|AEROSUL TÁXI AÉRE...|     Caçador Airport|       SBCD|   Santa Catarina|Hercílio Luz Inte...|                  SBFL|      Santa Catarina|
|ASTA LINHAS AÉREA...|Marechal Rondon I...|       SBCY|      Mato Grosso|       Juína Airport|                  SWJN|State of Mato Grosso|
|AZUL CONECTA LTDA...|       Coari Airport|       SWKO|State of Amazonas|Eduardo Gomes Int...|                  SBEG|            Amazonas|
|AZUL LINHAS AÉREA...|Santo